In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
import fitz  # PyMuPDF
## think i dont need any more ## from pathlib import Path

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set WD
import os 
os.getcwd()
# change path to the ma-rag-pipeline directory 
os.chdir("/workspaces/llm-zoomcamp/ma-rag-pipeline")

### 1. Import new entity and utility functions. 

In [3]:
from src.entities.user import load_users, get_user_by_id
from src.entities.spiritual_profile import load_spiritual_profiles, get_profile_by_user_id
from src.entities.conversation import load_conversations, append_conversation
from src.entities.persona import load_personas, get_persona_by_id
from src.utils.io import load_json, save_json, append_json_record
from pathlib import Path # I was already using this 

### 2. Set Paths

In [4]:

user_path = Path("data/users.json")
persona_path = Path("data/personas.json")
spiritual_profile_path = Path("data/spiritual_profiles.json")
conversation_path = Path("data/conversations.json")

### 3. Load Data

In [5]:
users = load_users(user_path)
spiritual_profiles = load_spiritual_profiles(spiritual_profile_path)
personas = load_personas(persona_path)
conversations = load_conversations(conversation_path)

In [6]:
user_path

PosixPath('data/users.json')

['Bob is struggling with marijuana addiction.',
 'He turned to marijuana as a coping mechanism after the loss of his cat and family.',
 'He feels betrayed by religion but is open to exploring spirituality.']